# CSI4142 Assignment 1

1. Andrew Pham - 300226985
2. Kevin Yao -

In [3]:
# import libraries

import pandas as pd
import numpy as np

## Dataset 1 - Google Play Store Apps

### Description
- Name: Google Play Store Apps
- Author: Lavanya Gupta
- Purpose: Measures the happiness of countries scored according to economic production, social support, etc., over the years (2015-2019)
- Shape: 10841 rows, 13 columns
- Features:
    - `App`: quasi-unique identifier, represents the application name
    - `Category`: categorical feature, represents the category the app belongs to
    - `Rating`: numerical feature, represents the average user rating of the app (1.0-5.0)
    - `Size`: quasi-numerical feature, represents the size of the app on disk
    - `Installs`: numerical feature, represents the number of installs the app has
    - `Type`: categorical feature, represents whether the app is paid or free
    - `Price`: numerical feature, represents the price of the app
    - `Content Rating`: categorical feature, represents the target age group
    - `Genres`: pseudo-categorical, represents the genre(s) an app belongs to. An app can belong to multiple genres.



[https://www.kaggle.com/datasets/lava18/google-play-store-apps](https://www.kaggle.com/datasets/lava18/google-play-store-apps)

In [17]:
gp_df = pd.read_csv(
    "https://raw.githubusercontent.com/krishnaik06/playstore-Dataset/refs/heads/main/googleplaystore.csv"
)
gp_df.shape

(10841, 13)

In [ ]:
print(
    f"Number of rows with duplicate app names: {sum(gp_df.duplicated(subset=['App']))}"
)

print(f"Number of exact duplicates: {sum(gp_df.duplicated())}")

# drop exact duplicates
gp_df = gp_df.drop_duplicates()

print(f"Shape after dropping exact duplicates: {gp_df.shape}")

SyntaxError: incomplete input (1270121615.py, line 10)

There are 1181 records that have a duplicated name, and 483 exact duplicate rows.

(10358, 13)